# Klasifikasi Limbah

------

## Data Cleaning

In [1]:
import os
import numpy as np
import pandas as pd
import lxml.html as lh

In [2]:
# Bagian buat ngebuka / ngebaca dataset "data-limbah.html"
# dataset diluar folder repository soalnya mengandung privasi

# Ganti variable path sesuai dengan lokasi dataset
# dataset ditaro di variable table
path = "/Users/bunny/myGit/DataStore/Limbah/"
table = ""

for ls in os.listdir(path):
    if ls.endswith(".html"):
        print (ls)
        with open(path + ls, 'r') as file:
            table = file.read()

data-limbah.html


In [3]:
# convert string ke html
doc = lh.fromstring(table)

# Ekstrak baris pada table
# <tr>
tr = doc.xpath("//tr")

In [4]:
# Ekstrak nama kolom di row pertama
# nama kolom dimasukin kw variable feature_name
feature_name = []

for td in tr[0]:
    feature_name.append(td.text_content())

# Posisi kolom yang digunakan [2, 13, 14, 15, 17]
# Nama Pengirim, Kode Limbah, Jenis Limbah, Karakter Limbah, Kemasan
feature_name = [feature_name[2]] + feature_name[13:16] + [feature_name[17]]

In [5]:
# Ekstrak data
data = []

for i in range(1, len(tr[1:])):
    tr_ = tr[i]
    ccol = 0
    data_tmp = []
    
    for td in tr_:
        # Filter kolom yang digunakan
        if ccol == 2 or (ccol >= 13 and ccol <= 17) and ccol != 16:
            data_tmp.append(td.text_content())
        
        # Menghitung kolom
        ccol += 1
    
    data.append(data_tmp)

In [6]:
# Cek jumlah data
len(data)

10269

In [7]:
# Mumbuat dataframe
df = pd.DataFrame(data, columns=feature_name)

In [8]:
# Menghapus duplikat berdasarkan Nama Pengirim dan Kode Limbah
# Nama Pengirim memiliki banyak data dengan Kode Limbah yang sama
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 5 columns):
Nama Pengirim      1020 non-null object
Kode Limbah        1020 non-null object
Jenis Limbah       1020 non-null object
Karakter Limbah    1020 non-null object
Kemasan            1020 non-null object
dtypes: object(5)
memory usage: 39.9+ KB


In [9]:
# Menghapus kolom Nama Pengirim (privasi)
df = df.drop(["Nama Pengirim"], axis=1)

In [10]:
# Sampel hasil ekstraksi
df.sample(3)

,Kode Limbah,Jenis Limbah,Karakter Limbah,Kemasan
591,B107d,Limbah elektronik termasuk cathode ray tube (C...,Beracun (toxic - T),Drum Plastik
428,B104d,Kemasan bekas B3,Beracun (toxic - T),Kemasan Karton atau plastik
851,B409,Fly ash,Beracun (toxic - T),"Karung Kain, plastik, kertas"


In [11]:
# Load dataset kode berdasarkan PP
kode = pd.read_csv("./Kode/kode-limbah.csv")
kode.sample(3)

,Kode,Sumber,Bahaya
627,A326-2,Spesifik Umum,1
168,A2105,?,1
446,B301-7,Spesifik Umum,2


In [12]:
# Load dataset bentuk
bentuk = pd.read_csv("./Kode/bentuk.csv")
bentuk.sample(3)

,Jenis,Bentuk
62,Asam Kloridad. Yang Tidak Spesifik Lain:,Cair
65,Limbah cat,Cair
70,Residu dari proses pencucian,Cair


In [13]:
# Menggabungkan dataframe
df = df.join(kode.set_index("Kode"), on="Kode Limbah")
df = df.join(bentuk.set_index("Jenis"), on="Jenis Limbah")

In [14]:
# Mengubah nilai bentuk berdasarkan karakter
# Cairan Mudah menyala (ignitable - I) / Padatan Mudah menyala (ignitable - I)
# Beberapa tidak sesuai
df.loc[df["Karakter Limbah"].str.startswith("Cair"), "Bentuk"] = "Cair"
df.loc[df["Karakter Limbah"].str.startswith("Padat"), "Bentuk"] = "Padat"

In [15]:
# Menampilkan data dengan bentuk "Cair & Padat"
df[df["Bentuk"] != "Cair & Padat"]

,Kode Limbah,Jenis Limbah,Karakter Limbah,Kemasan,Sumber,Bahaya,Bentuk
0,B104d,Kemasan bekas B3,Beracun (toxic - T),Tanpa Kemasan,Tidak Spesifik,2,Padat
1,A323-3,Residu proses produksi yang meliputi manufactu...,Beracun (toxic - T),Tanpa Kemasan,Spesifik Umum,1,Padat
2,A345-2,"Sludge logam antara lain berupa serbuk, gram d...",Beracun (toxic - T),Tanpa Kemasan,Spesifik Umum,1,Padat
3,B105d,Minyak pelumas bekas antara lain minyak peluma...,Cairan Mudah menyala (ignitable - I),Drum Logam,Tidak Spesifik,2,Cair
4,B104d,Kemasan bekas B3,Beracun (toxic - T),Tanpa Kemasan,Tidak Spesifik,2,Padat
5,A323-2,Sludge proses produksi yang meliputi manufactu...,Beracun (toxic - T),Jumbo Bag,Spesifik Umum,1,Padat
6,B104d,Kemasan bekas B3,Beracun (toxic - T),Tanpa Kemasan,Tidak Spesifik,2,Padat
7,A345-2,"Sludge logam antara lain berupa serbuk, gram d...",Beracun (toxic - T),Box Besi,Spesifik Umum,1,Padat
8,B104d,Kemasan bekas B3,Beracun (toxic - T),Tanpa Kemasan,Tidak Spesifik,2,Padat
9,A345-2,"Sludge logam antara lain berupa serbuk, gram d...",Beracun (toxic - T),Tanpa Kemasan,Spesifik Umum,1,Padat


In [16]:
# Mengubah nilai "Karakter Limbah"
# Cairan Mudah menyala (ignitable - I) / Padatan Mudah menyala (ignitable - I)
# Menjadi "I"
df["Karakter Limbah"] = df["Karakter Limbah"].str[-2]

In [17]:
# Filter dan mengurutkan kolom sesuai
df = df[["Kode Limbah", "Bentuk", "Karakter Limbah", "Sumber", "Bahaya", "Kemasan"]]
# df = df[["Kode Limbah", "Karakter Limbah", "Sumber", "Bahaya", "Kemasan"]]

In [18]:
# Mengganti tipe kolom (disesuaikan)
df = df.astype("category")
df["Kode Limbah"] = df["Kode Limbah"].astype("object")

In [19]:
# Export dataframe menjadi csv
df.to_csv("./data/limbah.csv", index=False)

## Data Exploration

In [20]:
df.sample(3)

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
684,B409,Padat,T,Spesifik Khusus,2,Tanpa Kemasan
868,A337-1,Cair,X,Spesifik Umum,1,Drum Plastik
373,B110d,Padat,T,Tidak Spesifik,2,Drum Logam


In [21]:
df["Bentuk"].value_counts()

Padat           690
Cair            318
Cair & Padat     12
Name: Bentuk, dtype: int64

In [22]:
df["Karakter Limbah"].value_counts()

T    805
I    135
X     42
C     33
E      5
Name: Karakter Limbah, dtype: int64

In [23]:
df["Sumber"].value_counts()

Tidak Spesifik     534
Spesifik Umum      374
Spesifik Khusus    111
?                    1
Name: Sumber, dtype: int64

In [24]:
df["Bahaya"].value_counts()

2    631
1    389
Name: Bahaya, dtype: int64

In [25]:
df["Kemasan"].value_counts()

Drum Logam                      287
Tanpa Kemasan                   218
Jumbo Bag                       153
Kemasan Karton atau plastik     122
Box Besi                         51
Tangki                           40
Karung Kain, plastik, kertas     39
Drum Plastik                     34
IBC Tank                         26
R.O Container                    20
Jerycan                          12
Wheel Bin                        10
Drum kayu                         5
Wadah Hopper                      1
Gerbong Barang                    1
                                  1
Name: Kemasan, dtype: int64

### Outlier

In [26]:
df[df["Sumber"] == '?']

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
800,A2358,Cair,T,?,1,Drum Logam


In [27]:
df[df["Kemasan"] == ""]

,Kode Limbah,Bentuk,Karakter Limbah,Sumber,Bahaya,Kemasan
982,B105d,Cair,I,Tidak Spesifik,2,


**Note**. 

1. Kolom bentuk punya 12 data bernilai "Cair & Padat" (hapus atau ubah)
2. Outlier nilai "?" dan kemasan tanpa nama "" muncul sekali (hapus atau ubah)
3. Nilai tidak seimbang

## Classification